# Example Notebook for Correcting and loading FastCCD Images

This example reads data from the FastCCD and corrects and rotates the images. Included are some
example of widgets to interact with the data

## Getting Started

Load the ``databroker`` moudle, ``csxtools`` and various other dependencies

In [1]:
import numpy as np
from databroker import DataBroker, get_table, get_images, get_events
from csxtools.utils import get_fastccd_images, get_images_to_4D
from csxtools.ipynb import image_stack_to_movie, show_image_stack
%matplotlib notebook
from matplotlib import pyplot as plt
from filestore.handlers import HandlerBase
import h5py 

In [ ]:
class AreaDetectorHDF5TimestampHandler(HandlerBase):
    """
    Handler for data stored in one Dataset of an HDF5 file.
    Parameters
    ----------
    filename : string
        path to HDF5 file
    key : string
        key of the single HDF5 Dataset used by this Handler
    frame_per_point : integer, optional
        number of frames to return as one datum, default 1
    """
    specs = {'AD_HDF5_TS'} | HandlerBase.specs
    
    def __init__(self, filename, frame_per_point=1):
        self._fpp = frame_per_point
        self._filename = filename
        self._key = ['/entry/instrument/NDAttributes/NDArrayEpicsTSSec',
                     '/entry/instrument/NDAttributes/NDArrayEpicsTSnSec']
        self._file = None
        self._dataset1 = None
        self._dataset2 = None
        self.open()

    def __call__(self, point_number):
        # Don't read out the dataset until it is requested for the first time.
        if not self._dataset1:
            self._dataset1 = self._file[self._key[0]]
        if not self._dataset2:
            self._dataset2 = self._file[self._key[1]]
        start, stop = point_number * self._fpp, (point_number + 1) * self._fpp
        rtn = self._dataset1[start:stop].squeeze()
        rtn = rtn + (self._dataset2[start:stop].squeeze() * 1e-9)
        #rtn = rtn.reshape(1,1, len(rtn))
        return rtn
    
    def open(self):
        if self._file:
            return
        self._file = h5py.File(self._filename, 'r')

    def close(self):
        super(AreaDetectorHDF5TimestampHandler, self).close()
        self._file.close()
        self._file = None
    

Set the logger level to info to see logging output, if required!

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

## Get the data headers from the DataBroker!

In [ ]:
bgnd = DataBroker['ce5042b7-00ae-49ca-9b11-b10c5661aaaa'] #[52440]

## Get the timestamps

In [ ]:
img = [i for i in get_events(bgnd, ['fccd_image_lightfield'], handler_overrides={'fccd_image_lightfield':AreaDetectorHDF5TimestampHandler})]

In [ ]:
timestamps = img[0]['data']['fccd_image_lightfield']

In [ ]:
diff = np.diff(timestamps)
fig, axes = plt.subplots(1)
n = axes.hist(diff, bins=100)

In [ ]:
timestamps

In [ ]:
import csxtools
csxtools.__version__

In [ ]:
import filestore
filestore.__version__

In [ ]:
import databroker
databroker.__version__